In [1]:
import torch
from flow_matching.supervised.alphas_betas import LinearAlpha, LinearBeta
from flow_matching.supervised.samplers import WHARSampler
from flow_matching.supervised.prob_paths import GaussianConditionalProbabilityPath
from flow_matching.supervised.models import SimpleUNet
from flow_matching.supervised.training import CFGTrainer

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
sampler = WHARSampler()

Creating config hash...
Checking download...
Download exists.
Checking sessions...
Sessions exist.
Validating common format...
[########################################] | 100% Completed | 731.71 ms
Common format validated.
Checking windowing...
Loading config hash...
Windowing exists.
subject_ids: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]
activity_ids: [0 1 2 3 4 5]
train: 16905 | val: 1878 | test: 3625
Postprocessing...


Loading windows: 100%|██████████| 22408/22408 [00:09<00:00, 2455.65it/s]


Getting normalization parameters...
Loading config hash...
Creating normalization parameters hash...
Loading normalization parameters hash...


Loading samples: 100%|██████████| 22408/22408 [00:01<00:00, 16394.79it/s]


In [4]:
shape = sampler.get_shape()
print(shape)
num_classes = len(sampler.dataset.get_class_weights(sampler.train_loader).keys())
print(num_classes)

path = GaussianConditionalProbabilityPath(
    p_data=sampler,
    p_simple_shape=shape,
    alpha=LinearAlpha(),
    beta=LinearBeta(),
).to(device)

unet = SimpleUNet(
    num_classes=num_classes,
    t_embed_dim=40,
    y_embed_dim=40,
).to(device)

trainer = CFGTrainer(path=path, model=unet, null_class=num_classes, eta=0.1)

[3, 4, 50]
6


In [10]:
trainer.train(num_epochs=1000, device=device, lr=1e-3, batch_size=32)

Training model with size: 0.425 MiB


Epoch 999, loss: 0.955: 100%|██████████| 1000/1000 [00:51<00:00, 19.52it/s]


In [11]:
torch.save(unet.state_dict(), "unet.pt")